In [1]:
# -*- coding: utf-8 -*-
import os
import sys
import math
import re
import pickle
import codecs
import numpy as np
import tensorflow as tf
from glob import glob
from tensorflow.models.rnn import rnn, rnn_cell
from tensorflow.python.framework import ops

In [2]:
epoch = 15 # epoch to train, 其实这个是指总的循环次数吗?
word_embed_dim = 650 # dimension of word embedding matrix 词向量矩阵
char_embed_dim = 15  # dimension of char embedding matrix 字向量矩阵
max_word_lenth = 65  # 词的最长长度, 这个对于中文没有必要好像...
batch_size = 100     # batch 最小的批处理
seq_length = 35      # no. of timesteps to unroll for ???
learning_Rate = 1.0
decay = 0.5  # decay of SGD
dropout_prob = 0.5   # probability of dropout layer, dropout layer到底是用在哪儿的???
feature_maps = [50,100,150,200,200,200,200] # no. of feature maps in CNN ???
kernels = [1,2,3,4,5,6,7]  # The width of CNN kernels ??? CNN的kernel 到底是啥???
#model = 'LSTMTDNN' # type of model to train and test, LSTM/LSTMTDNN
# data_dir
# dataset
# checkpoint_dir
forward_only = False # false for training
use_char = True  # use 字-lv 语言模型
use_word = False # use 词-lv 语言模型

In [3]:
# 写入
def save(fname, obj):
    with open(fname, 'w') as f:
        pickle.dump(obj, f)

# 读取
def load(fname):
    with open(fname, 'r') as f:
        pickle.load(f)

In [4]:
train_fname = 'data/wxb_gt_train.txt' # 黄金时代-> train
valid_fname = 'data/wxb_s_valid.txt'  # 沉默的大多数-> valid
test_fname = 'data/wxb_sf_test.txt'   # 思维的乐趣-> test
input_fnames = [train_fname, valid_fname, test_fname]

# vocab_fname = 'vocab.pkl'
# tensor_fname = 'data.pkl'
# char_fname = 'data_char.pkl'

In [5]:
# 处理 txt, 去掉空格空行
counts = []
for input_file in input_fnames:
    with codecs.open(input_file,'rw','utf-8') as f: # 用 codecs 打开 file 才不会出现编码问题
        a = f.read()
        a = a.replace(' ','') # 去掉空格
        a = a.replace('\n','') # 去掉空行
        print input_file,' with characters ',len(a) # 每个 txt 中有多少字
        # print a[:20]
        print input_file,'with unique characters ',len(set(a))
        #print 'sample of characters: ',list(set(a))[:10]

data/wxb_gt_train.txt  with characters  32443
data/wxb_gt_train.txt with unique characters  1829
data/wxb_s_valid.txt  with characters  9069
data/wxb_s_valid.txt with unique characters  1059
data/wxb_sf_test.txt  with characters  6215
data/wxb_sf_test.txt with unique characters  867


常用字: 一级常用字为3500, 二级常用字为3000字, 三级为姓氏人名地名等专有名词1605字, 总共8105字, http://www.gov.cn/gzdt/att/att/site1/20130819/tygfhzb.pdf

使用 HSK 字典, 总字数为2500常用字
https://github.com/OpenMindClub/OMAIWriter/wiki/ResDictionaries

In [11]:
#HSKc = pickle.load(codecs.open('data/HSKc.compatible.pickle', 'rb'))
obj=[]
for i in range(6):
    f = codecs.open('data/hsk_chars_L'+str(i+1)+'.txt','rb','utf-8')
    obj.append(f.read())

pickle.dump(obj, open('data/hsk_char.pkl','w'),2) # write to a file, can reuse

In [25]:
HSKc = pickle.load(codecs.open('data/hsk_char.pkl','rb')) 

In [26]:
print HSKc[0][:10] # 有\n 空行, 需要去除

爱
八
爸
杯
本



In [27]:
len(HSKc)

6

In [31]:
chars = []
for i in range(len(HSKc)):
    t = HSKc[i].split('\n')
    chars.append(t)

In [43]:
chars_all=[]
for li in chars:
    for c in li:
        chars_all.append(c)

In [44]:
len(chars_all)

2504

In [61]:
ooc = [] # out of character, 不在2500常用字典里
for input_file in input_fnames:
    with codecs.open(input_file,'rw','utf-8') as f:
        a = f.read()
        a = a.replace(' ','') 
        a = a.replace('\n','')
        a = list(set(a))
        #print a[10]
        for i in range(len(a)):
            if a[i] not in chars_all:
                ooc.append(a[i])

In [62]:
len(ooc)

385

In [67]:
for i in range(385):
    print ooc[i],

黝 蚯 罪 呸 》 酡 孽 芭 玷 甸 槛 裆 泞 薅 砂 坎 恬 秽 尿 撅 钥 薯 澳 汊 玻 秧 氓 泵 缕 粪 傣 窿 腮 裳 凹 妾 葫 喋 唠 墩 . 琶 卤 泱 蚂 瞌 耙 碾 凸 崽 蚁 粽 篓 蒂 爹 襟 瓢 寨 籽 箍 溯 蕉 、 匪 膛 蜴 屎 赘 彤 哩 1 赃 勒 叼 褂 “ 剐 寡 啪 睾 胳 乃 肝 笠 2 撵 Ｘ 豚 笋 吕 ” 筋 呐 苟 蹬 绷 匕 3 嚼 糠 潭 拴 沁 ， 爪 淫 奸 蝇 绢 咧 芳 4 腊 毙 袄 膊 l 垦 懊 弛 哼 ） 鸶 咒 棚 啧 叨 璃 蟥 蜥 蛤 浩 咙 烁 ？ ! 黏 a 坯 窑 籁 疚 耷 匠 犁 。 （ 皙 抡 炭 袒 腌 硝 藤 庇 骟 耸 赦 驴 捺 郭 镀 婊 搡 伧 9 y 刘 褪 莹 浒 哗 d 腺 刃 匙 倚 涧 唬 洼 矫 苹 ； 吭 掸 拎 ： 钳 莽 韵 缅 涎 窟 蹋 腕 脐 怦 撩 俏 靴 拽 蔓 戽 蜿 囊 哞 蚓 哟 疟 毋 杨 阉 掐 芦 跤 臼 唆 蜒 犟 铛 敦 跷 燎 揪 裸 妃 蓑 凳 藐 罗 鹭 — 哇 ！ 耿 《 铝 炊 擒 嗜 搀 箍 ： 贞 藉 癔 盥 拷 》 灶 懦 诲 暧 侏 髦 屎 柯 、 钎 1 ？ 啃 亢 儒 “ 喇 · 9 疙 叭 狐 w 。 （ 皙 宛 亩 佐 r 瘩 昧 迸 窟 芯 窿 u 募 嗡 捺 螂 s 艾 6 喋 ， 辄 奸 揪 t ” i 痰 — 冥 4 哇 腊 d ！ 莎 蒜 萧 崽 o 狸 蚁 《 ） 篓 吏 蒂 ； 吭 嗜 0 钳 7 罪 黯 扼 》 ： 盔 奸 艾 浩 迪 孽 、 邦 1 菲 儒 “ · 9 寡 檐 乃 莱 。 （ 垣 2 虞 ； 暇 ） ？ 屠 焉 丟 黜 孵 ， 淫 ” 爵 懈 兹 罗 赫 仲 — 4 尼 ！ 莎 萧 《 茨 徐 邪 凄 恕


观察不在字典表中的字, 似乎有不少也是常见的字, 这个字典表2500字似乎还是略小.....

在实现char-level 的[代码](https://github.com/carpedm20/lstm-char-cnn-tensorflow)中, 会将 txt 数据转化为 matrix时, 每一行为一个一个单词, 而 col 数目即为所有的字母, 对应中文应该怎样处理? col 数目假定为字典数目, 至少3000, 那每一行是什么? 也是词? 那么还是要分词吗? 还是取词典数量? 词典数量过大, 似乎还是分词靠谱一些? 不太理解啊